# Objective functions and optimization

The key idea to train neural networks is to change the network's parameters so that a certain objective function, also called loss function, is minimized. This is usually done by evaluating the gradient of this objective function with respect to the network's parameters. Actually being differentiable is therefore a key requirement for a useful objective function. How errors are weighted depends on the kind of learning problem, however, and can dramatically impact neural network performance.

### Loss functions for regression tasks
So far, we have considered the so-called <i>Mean-squared Error</i> (MSE)

$$ E=\frac{1}{2N}\sum_{i=1}^{N}(\hat{y_i}(w)-y_i)^2 $$

which is the average error over a set of $N$ pairs of predictions $\hat{y}$ that are dependent on the network parameters $w$ and known values $y$. This function is particularly convenient, as the square makes it convex, allowing to find its minimum by following its gradient ("gradient descent"), which we have seen for the backpropagation algorithm in the last chapter.

MSE is most suited for <i>regression</i> tasks in which data points are fitted to a model such as a line. Using a sigmoid or other continuous activation function, the error for each class can also be interpreted as a distance from the seperating hyperplane, which makes MSE also suitable (but not optimal for these kind of tasks). The two possible interpretations are illustrated below:

<center>
    <img src="figs/outliers.svg" width="30%">
</center>

Considering only the red dots, we could imagine the line being the MSE-best, that is the model that minimizes the MSE, to the red data points. Considering both red and blue dots, dots that are on the "right" side of the line will not contribute to the error, whereas those on the "wrong" side, such as the isolated red dot will contribute quadratically to the error, in particular when using an activation such as Relu that is not bound to one.

MSE therefore only poorly deals with outliers. If one value deviates largely from the prediction, the quadratic term in MSE will heavily punish this value. An alternative to MSE is the <i>Mean Absolute Error</i> (MAE):

$$E=\frac{1}{2N}\sum_{i=1}^N\|\hat{y_i}(w)-y_i\|) $$

Here, the absolute value $\|\dot\|$ ensures that the error is always positive, no matter the direction, but large errors have the same weight as smaller ones. MAE is therefore better suited if your trainingset contains outliers.

In practice, a large variety of loss functions are used that combine features of both MSE and MAE, in the simplest form by using a piecewise combination, such as the <i>Huber Loss</i> function. Some of these are well illustrated in the article <a href="https://heartbeat.fritz.ai/5-regression-loss-functions-all-machine-learners-should-know-4fb140e9d4b0">5 Regression Loss Functions All Machine Learners Should Know</i>.

In keras, the loss functions discussed here are available as <code>mean_squared_error</code>, <code>mean_absolute_error</code>, and <code>huber_loss</code>.

### Loss functions for classification tasks

Although a classification task can be cast into a regression problem, classifying is a lot more like throwing a dice. Indeed, the output of the Softmax layer is a discrete probability distribution in which each element $y_i=(p_0, \dots, p_c, \dots ,p_N)$ is the probability of an instance $x_i$ to be of class $c$ in $N$ classes total.

We speek of the <i>entropy</i> of a probability distribution as the amount of variety that we can expect. An uniform distribution therefore has the highest entropy - with a lot of possible outcomes, whereas data with which we train the network, the one-hot encoded vectors are probability distributions with very low entropy. The entropy of the distribution of $y_i$ (the training vector that stores the true class $c$ for each instance $x_i$) is given by 

$$ H(y_i)=-\sum_{c=1}{N}p_c \log p_c $$

Here, the logarithm can be of basis ten or two. In any case, the entropy function has a couple of interesting properties: first, the logarithm from 0 (negative infinity) to 1 is negative. This is why probabilities yield positive values. Second, the logarithm of 1 is zero, that is, a distribution with only one $p_c=1$ has the lowest possible entropy, zero. Third, the lower the individual entries for $p_c$ get, for example in an uniform distribution where $p_c=\frac{1}{N}$, the entropy is highest. <a href="https://towardsdatascience.com/entropy-is-a-measure-of-uncertainty-e2c000301c2c">This</a> is a good writeup on entropy. 

In every dataset, there will be a true distribution $P(C=i)$ that the data is distributed by. By classifying every element in the training set, the neural network also generates a distribution. Ideally, in the case of a 100% fit, the neural network will generate the exact same distribution as that of the training set. In the worst case, the network will generate a distribution that is completely different. <i>Evaluating a neural network's performance is therefore a matter of comparing two probability distributions.</i>

One way to compare two distributions is via their entropy, this is known as <i>cross entropy</i>:

$$ H(\hat{y},y)=-\sum_{i=1}{N}y_i\log \hat{y_i} $$

with $y_i=p_i$ the known probability for instance $x$ to be class $i$ and $\hat{y_i}$ the prediction. As the neural network will never perfectly represent the data, the cross entropy will allows be larger than the entropy of the true distribution, that is

$$ H(y)-H(\hat{y},y) \leq 0 $$

This difference between the entropy of the true distribution and the cross-entropy between the true and the estimated distribution is known as <i>Kullback-Leibler Divergence</i>. It is a measure of dissimilarity between two distributions. For further reading, the article <a href="https://towardsdatascience.com/understanding-binary-cross-entropy-log-loss-a-visual-explanation-a3ac6025181a">"Understanding binary cross-entropy and log-loss, a visual explanation"</a> is highly recommended. (This loss function is available as <code>kullback_leibler_divergence</code>. 

#### Binary and Categorical cross-entropy
In case there are only two classes, the <i>binary cross-entropy</i> is calculated as follows:

$$ H(\hat{y},y)=-\sum_{i=1}^Ny_i\log(\hat{y_i})=-y_1\log(\hat{y_1})-(1-y_1)\log(1-\hat{y_1})$$

As there are only two classes, true or false, $\hat{y_2}$ directly follows from $1-\hat{y_1}$. 
The more general case for $N>2$ is known as <i>categorical cross-entropy</i>. 

When using one-hot encoding, only class $c$ has probability 1 ($y_c=1$), reducing the cross-entropy to 

$$ H(\hat{y},y)=-\log(\hat{y_c}) $$

with $c$ the true class (the other terms are zero). 
combined with the softmax activation function the categorical cross entropy therefore computes as

$$ H(\hat{y},y) = -\log\left(\frac{e^{\hat{y_c}}}{\sum_{j}^N e^{\hat{y_j}}}\right) $$

It is available as <code>categorical_crossentropy</code> in keras. 





## Optimization

The key approach to training a neural network remains gradient descent. Gradient descent requires calculating (or estimating) the gradient of the objective function with respect to all weights of the network, a process usually done using the backpropagation algorithm. Once the gradient is known, it can be used to move toward the direction of the next extreme value. There are a couple of challenges with this approach

- the loss function is potentially highly non-linear, letting the algorithm get stuck in a local minima
- if moving too fast, the optimization algorithm might keep jumping over relevant minima
- if moving too slow, the optimization might not be able to finish in reasonable time

Both the simple perceptron training (Lecture 1) and backpropagation algorithms (Lecture 2) are naive implementations that avoid these issues only by fine-tuning the learning rate. The general form of gradient descent is

$$ dw=-\alpha \frac{\partial E}{\partial w}E(w)=-\alpha \sum_{i=1}^N\frac{\partial E_i}{\partial w}E(w)$$

with $E(w)$ the loss function, $w$ the network parameters, $\alpha$ the learning rate and $N$ the number of training examples. Note that a naive approach averages $dw$ over the entire training set, which is computationally expensive while progress toward the optimum is slow.





### Stochastic Gradient descent

It turns out that valid information on how to change the weights can already be gained from a single training example. What works well for that one does not necessarily works for others, leading in a rather rough exploration. In order to not repeat the same back-and-forth steps in each epoch, the training set is randomly shuffled. This approach is known as <i>stochastic gradient descent</i> (SGD). It turns out that a good trade-off is to estimate the gradient $\frac{\partial E}{\partial w}$ using a small subset of the training set, which is known as a <i>mini batch</i>.

Most SGD implementations use a <i>momentum term</i> to reduce oscillations and increase convergence speed. This is accomplished by updating the weights not only in the direction that the current gradient points, but also in that in which the weights have already been moving

$$ dw(k)=-\gamma dw(k-1) -\alpha \frac{\partial E}{\partial w}E(w) $$

Here $dw(t-1)$ is the change in $w$ one iteration earlier. A refinement of this idea is known as <i>Nesterov accelerated gradient</i> (NAG). It uses the fact that we already know where the $w$ will be going before even calculating the gradient as the momentum term (a fraction of the rate of change from the last iteration) is known. So why not calculating the gradient at this position instead?

$$ dw(k)=-\gamma dw(k-1) -\alpha \frac{\partial E}{\partial w}E(w-\gamma dw(k-1)) $$

This approach supposedly works well for recurrent neural networks (RNN), and is available via the <code>nesterov=true/false</code> option in the keras <code>SGD()</code> implementation.


### Adaptive gradient descent algorithms

SGD is computationally more efficient and faster than a naive approach that averages over all gradients in the training set, but it still requires hyperparameters such as learning rate and momentum. It also does not solve the problem of the learning rate trade-off that either learns too slow or missess important points in the fitness landscape. There exist a rich set of optimization algorithms that address these problems by adapting the gradient on the fly, such as Adagrad, Adadelta, RMSprop, Adam, Nadam and Adamax, all of which are available in Keras. 

<i>Adagrad</i> does not only adapt the learning rate, but maintains and updates different learning rates for each weight. Here, the learning rate for each weight is divided by the sum of squares of all previous gradients, decreasing the learning rate more as a function how much it has already changed. As learning would eventually stop <i>Adadelta</i> and <i>RMSprop</i> have proposed to divide the learning rate by a decaying running average of the squares of all previous gradients. Adam further extends these ideas by using a decaying average of all previous gradients as a momentum term. Please refer to the excellent article on <a href="https://ruder.io/optimizing-gradient-descent/">An overview of gradient descent algorithms</a> for more information and some insightful visualizations.

### Practical considerations

In practice, there are little differences between the different algorithms and Adam appears to have become the optimizer of choice for many. As whether a specific algorithm is better than another is often due to specific properties of the training data or the underlying generating functions. When trying to push things to an optimum it is therefore often worthwhile to use a slower method like SGD to see whether better results can be obtained by slow training. 

